# Ipyreact Tutorial

Welcome to this ipyreact walkthrough.

# Here you will learn:
* How to use the %react cell magic
* How to write a widegt
* How to load css for your widget
* how to add parameters to your widgets
* how to interact with these parameters

The tuturial will be based on a very simple react button to show all the ipyreact features.  
First, we will use the `%react` magic from ipyreact.  
The following line registers the cellmagic:

In [1]:
%load_ext ipyreact

In [2]:
%%react

import * as React from "react";

export default function Square() {
  return (<button> X </button>);
  }

Widget()

That worked! Next, we convert this into a widget.
For that, we need the code in a _ems string inside a class that inherits from `ipyreact.ReactWidget`

In [3]:
import ipyreact

class MyExampleWidget(ipyreact.ReactWidget):
    _esm = """
    import * as React from "react";

    export default function Square() {
        return <button> X </button> 
        };"""
MyExampleWidget()

MyExampleWidget()

now it's time to make some styling using CSS

In [4]:
from IPython.display import HTML
css_rules = """
button {
    background-color: yellow;
    border-radius: 10px;
    }
"""
HTML('<style>' + css_rules + '</style>')

You can also load CSS from a file.  
And you can add extra CSS without overwriting the previous CSS import.

In [5]:
from IPython.display import HTML
from pathlib import Path
css_rules = Path('styles_orange.css').read_text()
HTML('<style>' + css_rules + '</style>')

Note: When you clear the output of the previous cell, this CSS style will be removed from the notebook.

Next, we want to add parameters to the widget.  
First naive approach: Using f-strings.
That works, but is bad for two reasons:
1. curly brackets from f-strings will colide with the curly brackets from css. 
Using curly brackets in an f-string as a normal character can be done by doubling them like this : `{`to `{{`.
2. It is not possible to pass this parameter into the `MyExampleWidget` class, as it does not have an init function.

In [6]:
import ipyreact

# ❌❌❌  WARNING: THIS CODEBLOCK IS NOT GOOD PRACTICE ❌❌❌ 
class MyExampleWidget(ipyreact.ReactWidget):
    my_message = "Hello World"
    _esm = f"""
    import * as React from "react";

    export default function Square() {{
        return <button> {my_message} </button> 
        }};"""
MyExampleWidget()

MyExampleWidget()

Instead, we can use traitlets.  
traitlets are variables that can be used both in javascript and python.  
Note that it has to be initilized at `Square({my_message})`

In [7]:
import ipyreact
import traitlets

# ⭐⭐⭐  This is good practice ⭐⭐⭐ 
class MyExampleWidget(ipyreact.ReactWidget):
    my_message = traitlets.Unicode("Hi There").tag(sync=True)
    _esm = """
    import * as React from "react";

    export default function Square({my_message}) {
        return <button> {my_message} </button> 
        };"""
MyExampleWidget()

MyExampleWidget()

In [8]:
# we can use this traitlets also as parameters
MyExampleWidget(my_message = "Super Button")

MyExampleWidget(my_message='Super Button')

Now comes an example of a button that changes the postion using traitlets

In [11]:
import ipyreact
import traitlets

# ⭐⭐⭐  This is good practice ⭐⭐⭐ 
class MyExampleWidget(ipyreact.ReactWidget):
    my_position = traitlets.Unicode("My Postition").tag(sync=True)
    _esm = """
    import * as React from "react";

    export default function Square({my_position}) {
        return <button> {my_position} </button> 
        };"""
MyExampleWidget()

MyExampleWidget()